In [11]:
import pandas as pd
import openai
from openai.embeddings_utils import get_embedding
import os

In [12]:
openai.api_key = "key"

In [ ]:
#load the data with news articles
news = pd.read_csv("MIND/news.tsv", sep='\t', header=None) #document with news description
news.columns =['ID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Title Entities', 'Abstract Entities']
news['Content'] = news['Title'].fillna('') + ' ' + news['Abstract'].fillna('')
news = news.drop(['URL', 'Title Entities', 'Abstract Entities', 'Title', 'Abstract'], axis=1)
news.head()

In [ ]:
#function transforms the values of column from text into embeddings
def hashable_column(column_value):
    if isinstance(column_value, dict):
        hashable_items = []
        
        for key, value in column_value.items():
            if key == 'Content':
                # Convert 'Content' value to embedding
                value = get_embedding(value, engine="text-embedding-ada-002")
            elif isinstance(value, dict):
                value = hashable_column(value)
            elif isinstance(value, list):
                print("Unhashable list value at key:", key, "with value:", value)
            elif isinstance(value, str):
                value = value.replace("\n", " ")
            hashable_items.append((key, value))
        return tuple(sorted(hashable_items))
    else:
        if isinstance(column_value, str):
            return get_embedding(column_value, engine="text-embedding-ada-002")
        else:
            return column_value

In [ ]:
#save news embeddings 
news['Content_emb'] = news['Content'].apply(hashable_column)
news.to_csv('embeddings/news_emb_final.csv', index=False)

In [ ]:
news = pd.read_csv("embeddings/news_emb_final.csv") #document with news content
news.head()